<a href="https://colab.research.google.com/github/gitHubAndyLee2020/Transformer_Seq2Seq_Ko_En_Bidirectional_Translator/blob/main/test_translation_module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
!unzip kr_en_translation_ted_talk_dataset.zip

In [ ]:
!pip install 'portalocker>=2.0.0'

In [ ]:
!pip install -U torchdata
!pip install -U spacy
!python3 -m spacy download en_core_web_sm
!python3 -m spacy download ko_core_news_sm

In [12]:
FILE_PATH = 'multitarget-ted/en-ko/raw/'

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
from torchtext.data.utils import get_tokenizer

def load_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    return [line.strip() for line in lines]

def evaluate_bleu(model_translated_file, actual_translated_file, ln):
    # Get tokenizer
    if ln == 'en':
        tokenizer = get_tokenizer('spacy', language='en_core_web_sm')
    elif ln == 'ko':
        tokenizer = get_tokenizer('spacy', language='ko_core_news_sm')
    else:
        print("Unsupported language")
        return

    # Load files
    model_translated_lines = load_file(model_translated_file)
    actual_translated_lines = load_file(actual_translated_file)

    # Check if lengths are equal
    if len(model_translated_lines) != len(actual_translated_lines):
        print("The number of lines in both files must be the same.")
        return

    bleu_scores = []

    for model_line, actual_line in zip(model_translated_lines, actual_translated_lines):
        # Tokenize text
        model_tokens = tokenizer(model_line.lower())
        actual_tokens = tokenizer(actual_line.lower())

        # Compute BLEU score
        bleu_score = sentence_bleu([actual_tokens], model_tokens)
        bleu_scores.append(bleu_score)

    avg_bleu = sum(bleu_scores) / len(bleu_scores)
    print(f'Average BLEU score: {avg_bleu * 100:.2f}')

In [20]:
for i in range(0, 1):
  # Example usage
  model_translated_file = f"translated/translated_en_to_ko_epoch_{i}.txt"
  actual_translated_file = FILE_PATH + "ted_test1_en-ko.raw.ko"
  ln = "ko"  # 'en' for English, 'ko' for Korean
  print("Testing translated English-to-Korean text:")
  evaluate_bleu(model_translated_file, actual_translated_file, ln)

  # Example usage
  model_translated_file = f"translated/translated_ko_to_en_epoch_{i}.txt"
  actual_translated_file = FILE_PATH + "ted_test1_en-ko.raw.en"
  ln = "en"  # 'en' for English, 'ko' for Korean
  print("Testing translated Korean-to-English text:")
  evaluate_bleu(model_translated_file, actual_translated_file, ln)

Testing translated English-to-Korean text:
Average BLEU score: 0.09
Testing translated Korean-to-English text:
Average BLEU score: 0.37
